In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torchvision.transforms import transforms as T
import torchvision
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

import torchvision.models as models
import urllib

In [34]:
IMG_DIM = 224
BATCH_SIZE = 128

mean_nums = [0.485, 0.456, 0.406]
std_nums = [0.229, 0.224, 0.225]

img_transforms_train = T.Compose([
    T.Resize((IMG_DIM, IMG_DIM)),
    T.ToTensor(),
    T.Normalize(mean_nums, std_nums)
])

img_transforms_test = T.Compose([
    T.Resize((IMG_DIM, IMG_DIM)),
    T.ToTensor(),
    T.Normalize(mean_nums, std_nums)
])

train_dataset = ImageFolder('./images/train/', img_transforms_train)
test_dataset = ImageFolder('./images/test', img_transforms_test) 

train_loader = DataLoader(train_dataset, BATCH_SIZE, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, BATCH_SIZE, shuffle=False, num_workers=0)

In [42]:
# загрузить resnet18

resnet18 = models.resnet18(pretrained=True, progress=True)
vgg16 = models.vgg16(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/telaust/.cache/torch/checkpoints/vgg16-397923af.pth


In [16]:
print(f"num of training images (2 classes): {len(train_loader.dataset)}")

num of training images (2 classes): 37568


In [36]:
num_classes = 2

resnet18.fc = nn.Sequential(nn.Linear(resnet18.fc.in_features,512),
                                  nn.ReLU(),
                                  nn.Dropout(),
                                  nn.Linear(512, num_classes))

# vgg16.fc = nn.Sequential(nn.Linear(vgg16.fc.in_features,512),
#                                   nn.ReLU(),
#                                   nn.Dropout(),
#                                   nn.Linear(512, num_classes))

In [29]:
resnet18.fc

Sequential(
  (0): Linear(in_features=512, out_features=512, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=512, out_features=2, bias=True)
)

In [41]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    
    return acc

In [37]:
# цикл обучения
def train(model, optimizer, loss_fn, train_loader, test_loader, n_epochs=10, device='cpu'):
    # для каждой эпохи
    for epoch in range(n_epochs):
        # иниц тренировчную потерю
        train_loss = 0.0
        # переключаем модель в режим тренировки
        model.train()

        # для каждого батча (пакета)
        for batch in train_loader:
            # очищаем вычислительный граф
            optimizer.zero_grad()

            # изымаем вход и целевые значения
            inputs, targets = batch
            #
            inputs = inputs.to(device)
            targets = targets.to(device)
            # получаем выход модели
            outputs = model(inputs)

            # считаем потерю
            loss = loss_fn(outputs, targets)
            #
            loss.backward()

            optimizer.step()
            #
            train_loss+= loss.data.item() * inputs.size(0)
            
        train_loss /= len(train_loader.dataset)
        print(f"epoch#{epoch}, train loss = {train_loss}")
        
        print("train ok")
        # ********* EVAL *********
        
        model.eval()
        val_loader = test_loader
        
        num_corrects = 0
        num_examples = 0
        val_loss = 0.0
        
        for batch in val_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            
            output = model(inputs)
            targets = targets.to(device)
            
            loss = loss_fb(output, targets)
            valid_loss += loss.data.item() * inputs.size(0)
            
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
            
        valid_loss /= len(val_loader.dataset)
        
        # вывод
        print(f"ep: {epoch}, train loss: {train_loss}, val loss: {val_loss}")
    

In [38]:
# заморозить все параметры, кроме пакетной нормализации
for name, param in resnet18.named_parameters():
    if("bn" not in name):
        param.requires_grad = False

In [32]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet18.parameters(), lr=1e-2)

In [39]:
train(resnet18, optimizer, criterion, train_loader, test_loader, n_epochs=3)

KeyboardInterrupt: 